In [1]:
# prompt: create a dataset with name 'emergency_calls_response'

from google.cloud import bigquery

client = bigquery.Client()
dataset_id = 'emergency_calls_response'

dataset_ref = client.dataset(dataset_id)
dataset = bigquery.Dataset(dataset_ref)
dataset.location = "US"
dataset = client.create_dataset(dataset)  # Make an API request.

print("Dataset {} created.".format(dataset.dataset_id))


Dataset emergency_calls_response created.


In [3]:
# prompt: import csv into bigquery table in above created dataset from 'gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv"

from google.cloud import bigquery

client = bigquery.Client()
dataset_id = 'emergency_calls_response'
table_id = 'emergency_calls_response_times'

# Construct a BigQuery client object.
client = bigquery.Client()

# Set table_id to the ID of the table to create.
# table_id = "your-project.your_dataset.your_table"

# Construct a reference to the "my_dataset" dataset
dataset_ref = client.dataset(dataset_id)

# Construct a BigQuery table object
table_ref = dataset_ref.table(table_id)
table = bigquery.Table(table_ref)

# Configure the load job.
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    skip_leading_rows=1,
    # The source format defaults to CSV, so the property is optional.
    source_format=bigquery.SourceFormat.CSV,
    write_disposition="WRITE_TRUNCATE"
)

# Specify the Cloud Storage URI of the file to load.
uri = "gs://labs.roitraining.com/data-to-ai-workshop/emergency_calls_response_times.csv"

# Start the load job.
load_job = client.load_table_from_uri(
    uri, table, job_config=job_config
)  # Make an API request.

print("Starting job {}".format(load_job.job_id))

load_job.result()  # Waits for the job to complete.

print("Job finished.".format(load_job.job_id))

destination_table = client.get_table(table_ref)
print(
    "Loaded {} rows into {}.".format(
        destination_table.num_rows, table_id
    )
)


Starting job 0e79d036-b8e4-4f1a-8f3e-6e2c2e212417
Job finished.
Loaded 50000 rows into emergency_calls_response_times.


In [7]:
query_1 = """SELECT * FROM `emergency_calls_response.emergency_calls_response_times`"""

emergency_calls_response_df = client.query(query_1).to_dataframe()
emergency_calls_response_df.head()

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37


In [8]:
emergency_calls_response_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   call_id              50000 non-null  Int64              
 1   call_timestamp       50000 non-null  datetime64[us, UTC]
 2   call_type            50000 non-null  object             
 3   location             50000 non-null  object             
 4   weather_condition    50000 non-null  object             
 5   day_of_week          50000 non-null  object             
 6   time_of_day          50000 non-null  Int64              
 7   traffic_level        50000 non-null  object             
 8   distance_to_station  50000 non-null  float64            
 9   units_available      50000 non-null  Int64              
 10  response_time        50000 non-null  float64            
dtypes: Int64(3), datetime64[us, UTC](1), float64(2), object(5)
memory usage: 4.3+ MB

In [9]:
emergency_calls_response_df.describe()


,call_id,time_of_day,distance_to_station,units_available,response_time
count,50000.0,50000.0,50000.000000,50000.0,50000.000000
mean,25000.5,11.5143,15.079754,7.9979,17.446134
std,14433.901067,6.930766,8.636711,4.315867,5.296881
min,1.0,0.0,0.100000,1.0,2.010000
25%,12500.75,6.0,7.620000,4.0,13.590000
50%,25000.5,12.0,15.080000,8.0,17.250000
75%,37500.25,18.0,22.560000,12.0,21.100000
max,50000.0,23.0,30.000000,15.0,36.550000


In [11]:
print(emergency_calls_response_df.shape)
print(emergency_calls_response_df.info())

(50000, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   call_id              50000 non-null  Int64              
 1   call_timestamp       50000 non-null  datetime64[us, UTC]
 2   call_type            50000 non-null  object             
 3   location             50000 non-null  object             
 4   weather_condition    50000 non-null  object             
 5   day_of_week          50000 non-null  object             
 6   time_of_day          50000 non-null  Int64              
 7   traffic_level        50000 non-null  object             
 8   distance_to_station  50000 non-null  float64            
 9   units_available      50000 non-null  Int64              
 10  response_time        50000 non-null  float64            
dtypes: Int64(3), datetime64[us, UTC](1), float64(2), object(5)
memory us

In [12]:
print(emergency_calls_response_df.isnull().sum())

call_id                0
call_timestamp         0
call_type              0
location               0
weather_condition      0
day_of_week            0
time_of_day            0
traffic_level          0
distance_to_station    0
units_available        0
response_time          0
dtype: int64


In [13]:
# Unique values for categorical columns
categorical_cols = ['call_type', 'location', 'weather_condition', 'day_of_week', 'traffic_level']
for col in categorical_cols:
    print(f"\n{col}: {emergency_calls_response_df[col].nunique()} unique values")
    print(emergency_calls_response_df[col].value_counts())


call_type: 4 unique values
call_type
Fire       12585
Police     12536
Medical    12478
Rescue     12401
Name: count, dtype: int64

location: 8 unique values
location
Uptown        6393
Greenfield    6273
Brookfield    6265
Highland      6255
Maplewood     6226
Downtown      6212
Riverside     6201
Oakmont       6175
Name: count, dtype: int64

weather_condition: 4 unique values
weather_condition
Sunny    12633
Snowy    12558
Rainy    12429
Windy    12380
Name: count, dtype: int64

day_of_week: 7 unique values
day_of_week
Saturday     7340
Thursday     7193
Friday       7178
Monday       7108
Wednesday    7104
Sunday       7054
Tuesday      7023
Name: count, dtype: int64

traffic_level: 3 unique values
traffic_level
High      16744
Medium    16655
Low       16601
Name: count, dtype: int64


In [16]:
emergency_calls_response_df.head()

,call_id,call_timestamp,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available,response_time
0,35957,2023-01-01 00:05:53+00:00,Fire,Highland,Rainy,Sunday,0,High,21.45,3,23.41
1,20832,2023-01-01 00:20:47+00:00,Fire,Oakmont,Rainy,Sunday,0,High,22.29,6,20.11
2,27949,2023-01-01 00:33:27+00:00,Fire,Riverside,Windy,Sunday,0,High,17.19,14,19.75
3,20199,2023-01-01 00:48:29+00:00,Fire,Riverside,Windy,Sunday,0,High,17.39,14,20.76
4,46938,2023-01-01 00:50:44+00:00,Rescue,Brookfield,Sunny,Sunday,0,High,22.50,14,22.37


In [14]:
%%bigquery

CREATE OR REPLACE MODEL `emergency_calls_response.emergency_response_model`
OPTIONS(
  model_type='LINEAR_REG',
  input_label_cols=['response_time']
) AS
SELECT
  call_type,
  location,
  weather_condition,
  day_of_week,
  time_of_day,
  traffic_level,
  distance_to_station,
  units_available,
  response_time
FROM `emergency_calls_response.emergency_calls_response_times`


Query is running:   0%|          |

""


In [15]:
%%bigquery

SELECT *
FROM ML.EVALUATE(
  MODEL `emergency_calls_response.emergency_response_model`,
  (SELECT
     call_type,
     location,
     weather_condition,
     day_of_week,
     time_of_day,
     traffic_level,
     distance_to_station,
     units_available,
     response_time
   FROM `emergency_calls_response.emergency_calls_response_times`)
)


Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.741854,4.73939,0.014721,1.479472,0.831076,0.831078


In [17]:
%%bigquery

CREATE OR REPLACE MODEL `emergency_calls_response.random_forest_model`
OPTIONS(
  model_type='RANDOM_FOREST_REGRESSOR',
  input_label_cols=['response_time']
) AS
SELECT
  call_type,
  location,
  weather_condition,
  day_of_week,
  time_of_day,
  traffic_level,
  distance_to_station,
  units_available,
  response_time
FROM `emergency_calls_response.emergency_calls_response_times`

Query is running:   0%|          |

""


In [18]:
%%bigquery

SELECT *
FROM ML.EVALUATE(
  MODEL `emergency_calls_response.random_forest_model`,
  (SELECT
     *
   FROM `emergency_calls_response.emergency_calls_response_times`)
)


Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.919169,5.844221,0.018128,1.60209,0.791697,0.791841


In [19]:
%%bigquery

-- Model 1: Linear Regression with AUTO_SPLIT
CREATE OR REPLACE MODEL `emergency_calls_response.linear_regression_model`
OPTIONS(
  model_type='LINEAR_REG',
  input_label_cols=['response_time'],
  DATA_SPLIT_METHOD='AUTO_SPLIT'  -- BigQuery automatically splits: 80% train, 20% eval
) AS
SELECT
  call_type,
  location,
  weather_condition,
  day_of_week,
  time_of_day,
  traffic_level,
  distance_to_station,
  units_available,
  response_time
FROM `emergency_calls_response.emergency_calls_response_times`;


Query is running:   0%|          |

""


In [21]:
%%bigquery

SELECT *
FROM ML.EVALUATE(
  MODEL `emergency_calls_response.linear_regression_model`,
  (SELECT
     *
   FROM `emergency_calls_response.emergency_calls_response_times`)
)

Query is running:   0%|          |

Downloading:   0%|          |

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,1.741854,4.73939,0.014721,1.479472,0.831076,0.831078


## **Make Prediction**

In [28]:
# Get unique values for each column as a dictionary
unique_per_col = {col: emergency_calls_response_df[col].unique() for col in emergency_calls_response_df.columns}

for col, values in unique_per_col.items():
    print(f"{col}: {values}")

call_id: <IntegerArray>
[35957, 20832, 27949, 20199, 46938, 17582, 21624, 36793, 41350, 32092,
 ...
  5727, 49107, 10405, 28281, 38841, 20813, 44525, 13740,  7395, 17910]
Length: 50000, dtype: Int64
call_timestamp: <DatetimeArray>
['2023-01-01 00:05:53+00:00', '2023-01-01 00:20:47+00:00',
 '2023-01-01 00:33:27+00:00', '2023-01-01 00:48:29+00:00',
 '2023-01-01 00:50:44+00:00', '2023-01-01 02:28:50+00:00',
 '2023-01-01 02:44:06+00:00', '2023-01-01 02:53:54+00:00',
 '2023-01-01 03:52:33+00:00', '2023-01-01 04:09:23+00:00',
 ...
 '2023-12-30 20:34:36+00:00', '2023-12-30 20:37:21+00:00',
 '2023-12-30 21:08:11+00:00', '2023-12-30 21:22:29+00:00',
 '2023-12-30 21:27:47+00:00', '2023-12-30 21:28:37+00:00',
 '2023-12-30 21:40:21+00:00', '2023-12-30 23:21:57+00:00',
 '2023-12-30 23:41:59+00:00', '2023-12-30 23:49:30+00:00']
Length: 49951, dtype: datetime64[us, UTC]
call_type: ['Fire' 'Rescue' 'Police' 'Medical']
location: ['Highland' 'Oakmont' 'Riverside' 'Brookfield' 'Downtown' 'Greenfield'
 'M

In [29]:
%%bigquery

SELECT *
FROM ML.PREDICT(
  MODEL `emergency_calls_response.emergency_response_model`,
  (SELECT
     'Medical' AS call_type,
     'Riverside' AS location,
     'Clear' AS weather_condition,
     'Monday' AS day_of_week,
     14 AS time_of_day,
     'Moderate' AS traffic_level,
     2.5 AS distance_to_station,
     3 AS units_available
  )
)


Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,286.19769,Medical,Riverside,Clear,Monday,14,Moderate,2.5,3


In [30]:
%%bigquery

SELECT *
FROM ML.PREDICT(
  MODEL `emergency_calls_response.linear_regression_model`,
  (SELECT
     'Medical' AS call_type,
     'Greenfield' AS location,
     'Snowy' AS weather_condition,
     'Monday' AS day_of_week,
     16 AS time_of_day,
     'Medium' AS traffic_level,
     4.5 AS distance_to_station,
     5 AS units_available
  )
)


Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,14.520853,Medical,Greenfield,Snowy,Monday,16,Medium,4.5,5


### Trying using same data

In [33]:
%%bigquery
SELECT *
FROM ML.PREDICT(
  MODEL `emergency_calls_response.emergency_response_model`,
  (SELECT
     'Fire' AS call_type,
     'Uptown' AS location,
     'Windy' AS weather_condition,
     'Saturday' AS day_of_week,
     1 AS time_of_day,
     'Low' AS traffic_level,
     12.5 AS distance_to_station,
     8 AS units_available
  )
)

Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,12.766055,Fire,Uptown,Windy,Saturday,1,Low,12.5,8


In [31]:
%%bigquery

SELECT *
FROM ML.PREDICT(
  MODEL `emergency_calls_response.random_forest_model`,
  (SELECT
     'Fire' AS call_type,
     'Uptown' AS location,
     'Windy' AS weather_condition,
     'Saturday' AS day_of_week,
     1 AS time_of_day,
     'Low' AS traffic_level,
     12.5 AS distance_to_station,
     8 AS units_available
  )
)


Query is running:   0%|          |

Downloading:   0%|          |

,predicted_response_time,call_type,location,weather_condition,day_of_week,time_of_day,traffic_level,distance_to_station,units_available
0,12.368058,Fire,Uptown,Windy,Saturday,1,Low,12.5,8
